In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
pd.options.display.float_format = '{:.2f}'.format

from itertools import combinations
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from prophet import Prophet

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Measles/benin.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Year-Month  159 non-null    object
 1   Cases       159 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.6+ KB


In [5]:
df.columns = ['ds','y']

In [6]:
train_data_pr = df.iloc[:len(df)-12]
test_data_pr = df.iloc[len(df)-12:]

In [7]:
len(test_data_pr)

12

In [8]:
test_data_pr

,ds,y
147,2023-4,63
148,2023-5,48
149,2023-6,26
150,2023-7,15
151,2023-8,20
152,2023-9,11
153,2023-10,16
154,2023-11,15
155,2023-12,28
156,2024-1,29


In [9]:
test_data_pr['y']

147    63
148    48
149    26
150    15
151    20
152    11
153    16
154    15
155    28
156    29
157    83
158    24
Name: y, dtype: int64

In [10]:
from enum import auto
#m = Prophet()
m = Prophet(seasonality_mode = "additive",yearly_seasonality=False,weekly_seasonality=False,daily_seasonality=False,
            changepoints=None, n_changepoints=5, changepoint_range=0.8)
m.fit(train_data_pr)
future = m.make_future_dataframe(periods=12,freq='MS')
prophet_pred = m.predict(future)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpwnja_7ln/13nnzdbi.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpwnja_7ln/nwcwy1e4.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=3781', 'data', 'file=/tmp/tmpwnja_7ln/13nnzdbi.json', 'init=/tmp/tmpwnja_7ln/nwcwy1e4.json', 'output', 'file=/tmp/tmpwnja_7ln/prophet_modelcwfh7upw/prophet_model-20240720071743.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
07:17:43 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:17:43 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [11]:
prophet_pred = pd.DataFrame({"Date" : prophet_pred[-12:]['ds'], "Pred" : prophet_pred[-12:]["yhat"]})

In [12]:
prophet_pred = prophet_pred.set_index("Date")

In [13]:
prophet_pred.index.freq = "MS"

In [14]:
prophet_pred

,Pred
Date,
2023-04-01,21.56
2023-05-01,21.42
2023-06-01,21.27
2023-07-01,21.13
2023-08-01,20.99
2023-09-01,20.84
2023-10-01,20.70
2023-11-01,20.56
2023-12-01,20.42


In [16]:
from sklearn.metrics import mean_squared_error
error = np.sqrt(mean_squared_error(test_data_pr['y'],prophet_pred['Pred']))
#print('Test RMSE: %.4f' % error)

def mean_absolute_percentage_error(y_true, y_pred):

    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


mape = mean_absolute_percentage_error(test_data_pr['y'],prophet_pred['Pred'])

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(test_data_pr['y'],prophet_pred['Pred'])
#print("Mean Absolute Error (MAE):", mae)

from sklearn.metrics import mean_squared_log_error
msle = mean_squared_log_error(test_data_pr['y'],prophet_pred['Pred'])
#print("Mean Squared Logarithmic Error (MSLE):", msle)

from sklearn.metrics import mean_absolute_percentage_error
#print('Mean Absolute Percentage Error (MAPE):',mean_absolute_percentage_error(test_data_pr['y'],prophet_pred['Pred']))

print('RMSE: %.2f' % error)
print(f"MAPE: {mape:.2f}%")
print('MAE:  %.2f' % mae)
print('MSLE: %.2f' % msle)

RMSE: 23.69
MAPE: 40.89%
MAE:  15.26
MSLE: 0.37
